In [1]:
import logging
import logging.config
import os
import json
# import ConfigParser
import sys

from simpleMenu import Menu
import configuration
from gdrive.auth import getCredentials
from gdrive.gdrive import googledrive, GDriveError


# get the current working directory
# When launching a .command from the OS X Finder, the working directory is typically ~/; this is problematic
# for locating resource files
# I don't love this hack, but it works.
try:
    __file__
    cwd = os.path.dirname(__file__)+'/'
except NameError as e:
    cwd = os.getcwd()

In [2]:
def setup_logging(
    default_path='logging.json',
    default_level=logging.INFO,
    env_key='LOG_CFG'
):
    """Setup logging configuration

    """
    path = default_path
    value = os.getenv(env_key, None)
    if value:
        path = value
    if os.path.exists(path):
        with open(path, 'rt') as f:
            config = json.load(f)
        logging.config.dictConfig(config)
        logging.getLogger().setLevel(default_level)
    else:
        logging.basicConfig(level=default_level)

In [3]:
def getConfiguration(cfgfile):
    # required configuraiton options
    # Section: {'option': 'default value'}
    cfgpath = os.path.dirname(cfgfile)
    config_required = {
        'Main': {'credentials': os.path.join(cfgpath, 'credentials/'), 
                 'teamdriveID': '',
                 'teamdrivename': '',
                 'folderID': '',
                 'foldername': '',
                 },
        }

    config = configuration.get_config(cfgfile)

    update_config = False

    for section, values in config_required.items():
        if not config.has_section(section):
            logger.warn('section: {} not found in {}'.format(section, cfgfile))
            logger.debug('adding section {}'.format(section))
            config.add_section(section)
            update_config = True
        for option, value in values.items():
            if not config.has_option(section, option):
                logger.warn('option: {} not found in {}'.format(option, cfgfile))
                logger.debug('adding option {}: {}'.format(option, value))

                config.set(section, option, value)
                update_config = True


    # for section, options in config_required.items():

    if update_config:
        try:
            logger.info('updating configuration file at: {}'.format(cfgfile))
            configuration.create_config(cfgfile, config)
        except Exception as e:
            logger.error(e)
            
    return(config)

In [4]:
def getTeamDrive(myDrive):
    '''
    menu interaction to find appropriate teamdrive ID for storing portfolios
    '''
    logger = logging.getLogger(__name__)
    myMenu = Menu()
    teamDrives = {}
    logging.debug('fetching list of team drives')
    for each in myDrive.listTeamDrives():
        try:
            teamDrives[each['name']] = each['id']
            myMenu.addItem(each['name'])
        except KeyError as e:
            logger.warn('KeyError: {}'.format(e))
            
    myMenu.sortMenu()
    td_name = myMenu.loopChoice(optional=True, message='Choose Team Drive for Portfolios')
    if td_name.upper() == 'Q':
        return('Q', 'Q')
    else:
        return([td_name, teamDrives[td_name]])

In [5]:
def XgetPortfolioFolder(myDrive, teamdriveID):
    '''
    menu interaction to find appropriate folder for storing portfolios
    '''
    logger = logging.getLogger(__name__)
    myMenu = Menu()
    folders = {}
    print 'Locating Portfolio Folder in Team Drive: {}'.format(teamdriveID)

    rawfilelist = []
    attempt = 3
    while (len(rawfilelist) == 0) and (attempt > 0):
        print 'tries: {}'.format(attempt)
        foldersearch = ''
        while len(foldersearch) < 1:
            foldersearch = raw_input('Enter part of the protfolio folder name (case sensitive search): ')
            
        logging.debug('searching TD: {} for {}'.format(teamdriveID, foldersearch))
        result = myDrive.search(name=foldersearch, fuzzy=True, teamdrive = teamdriveID, mimeType='folder')
        rawfilelist = result['files']

        if len(rawfilelist) == 0:
            print 'no folders were found that match: {}'.format(foldersearch)
        attempt -=1
    
    if len(rawfilelist) == 0:
        return('Q')
    
    filelist = {}
    for each_folder in rawfilelist:
        filelist[each_folder['name']] = each_folder['id']
        myMenu.addItem(each_folder['name'])
    
    myMenu.sortMenu()
    folder = myMenu.loopChoice(optional=True, optchoices= {'Q': 'Quit', 'N': 'New Search'}, 
                                message='Choose the folder to use from the list below')

    return(td_name)
        
        
    
    logger.debug('')


In [178]:
def getPortfolioFolder(myDrive, teamDriveID):
    # ask user for partial folder name
        # check for non 0 input
    # if results are returned ask user to choose from list, or search again
    # if no results are returned, prompt user for new search terms
    # return the name and id of the folder
    
    logger = logging.getLogger(__name__)
    logger.info('starting search for portfolio folder in teamdrive ID: {}'.format(teamdriveID))
    print 'Locating Portfolio Folder in Team Drive'
    
    search = True
    attempts = 3
    
    quitMenu = Menu(items=['Yes', 'No'])
    
    while search:
        logger.debug('remaining attempts: {}'.format(attempts))
        attempts -=1
        foldersearch = ''
        folderID = ''
        folderName = ''
        folderlist = {}
        myMenu = Menu()
        
        # get a partial file name to search for
        while len(foldersearch) <= 0:
            foldersearch = raw_input('Enter part of the portfolio folder name: ')
            if len(foldersearch) <=0:
                print 'Please enter at least one character.'
                
        # fetch results from google drive
        logger.debug('searching for folder name: {}'.format(foldersearch))
        searchresult = myDrive.search(name=foldersearch, fuzzy=True, teamdrive = teamdriveID, mimeType='folder')
        try:
            rawfilelist = searchresult['files']
        except KeyError as e:
            rawfilelist = {}
            logger.warn(e)
    
        if len(rawfilelist) < 1:
            print 'No matching folders were found. Please check your spelling and try a new search.'
            logger.debug('no matching folders found')
        else:
            for each_folder in rawfilelist:
                folderlist[each_folder['name']] = each_folder['id']
                myMenu.addItem(each_folder['name'])
                
            myMenu.sortMenu()
            folderName = myMenu.loopChoice()
            folderID = folderlist[folderName]
            if len(folderID) > 0:
                logger.debug('exiting folder choice loop')
                break
        
            
            
        # ask to break out of loop after attempts reaches 0
        if attempts <= 0:
            quit = quitMenu.loopChoice(message='It looks like you\'re having trouble. Would you like to try again?')
            if quit == 'No':
                search = False
            else:
                attempts = 3
        
    
    
    
    return(folderName, folderID)

In [187]:

version = '00.00 - 18.08.13'
appName = 'portfolio_creator'

cfgfile = appName+'.ini'
cfgpath = os.path.join('~/.config/', appName)
cfgfile = os.path.expanduser(os.path.join(cfgpath, cfgfile))
updateConfig = False

logger = logging.getLogger(__name__)
setup_logging(default_level=logging.DEBUG)
logging.info('===Starting {} Log==='.format(appName))

myConfig = getConfiguration(cfgfile)

####### set all config options 
if myConfig.has_option('Main', 'credentials'):
    credential_store = os.path.expanduser(myConfig.get('Main', 'credentials'))
else:
    credential_store = os.path.expanduser(os.path.join(cfgpath, 'credentials'))
    updateConfig = True

if myConfig.has_option('Main', 'teamdriveid'):
    teamdriveID = myConfig.get('Main', 'teamdriveid')
else:
    teamdriveID = None
    
if myConfig.has_option('Main', 'teamdrivename'):
    teamdriveName = myConfig.get('Main', 'teamdrivename')
else:
    teamdriveName = None
    
if myConfig.has_option('Main', 'folderid'):
    folderID = myConfig.get('Main', 'folderid')
else:
    folderID = None

if myConfig.has_option('Main', 'foldername'):
    folderName = myConfig.get('Main', 'foldername')
else:
    folderName = None

if myConfig.has_option('Main', 'gradefolders'):
    gradefolders = myConfig.get('Main', 'gradefolders')
else:
    gradefolders = None
    
if logging.getLogger().getEffectiveLevel() <= 10:
    config_dict = {}
    for section in myConfig.sections():
        config_dict[section] = {}
        for option in myConfig.options(section):
            config_dict[section][option] = myConfig.get(section, option)

    logging.debug('current configuration:')
    logging.debug('\n{}'.format(config_dict))

# print '\n'*10
print 'Welcome to the portfolio creator for Google Team Drive version {0}'.format(version)
print 'This program will create portfolio folders in Google Team Drive for students.'
print 'You will need a student_export.text file from PowerSchool with at least the following information:'
print '     ClassOf, FirstLast, Student_Number\n'
print 'The order of the CSV does not matter, but the headers must be on the very first line.'
# raw_input("Press Enter to continue...\n\n\n")

    
# google drive authorization
logging.info('checking google credentials')
#### Add instructions before this shoots user over to a web authorization 
print '\n'*3
print 'You *may* be directed to a google web page that asks you to authorize this applicaiton.'
print 'Please choose an ASH account and authorize this applicaiton.'
print 'When you are done, please close the newly created tab in your web browser and return to this window.'
raw_input("Press Enter to continue...\n\n\n")
credential_store = os.path.expanduser(myConfig.get('Main', 'credentials'))

# attempt to authorize using stored credentials or generate new credentials as needed
try:
    credentials = getCredentials(credential_store)
except Exception as e:
    logging.critical(e)

# configure google drive object
myDrive = googledrive(credentials)

if (not teamdriveID) or (len(teamdriveID)< 1):
    teamdriveName, teamdriveID = getTeamDrive(myDrive)
    if teamdriveID == 'Q':
        logging.debug('user exit')
        sys.exit(0)
    myConfig.set('Main', 'teamdriveid', teamdriveID)
    myConfig.set('Main', 'teamdrivename', teamdriveName)
    updateConfig = True


if not folderID or len(folderID) < 1:
    folderName, folderID = getPortfolioFolder(myDrive, teamdriveID)
    if len(folderID) < 1:
        logging.debug('user exit')
        print 'exiting'
    myConfig.set('Main', 'folderID', folderID)
    myConfig.set('Main', 'folderName', folderName)
    updateConfig = True


if not gradefolders or len(gradefolders) < 1:
    if os.path.isfile('./gradefolders.txt'):
        gradefolders = './gradefolders.txt'
        myConfig.set('Main', 'gradefolders', gradefolders)
        updateConfig = True
    if os.path.isfile(os.path.expanduser('~/Desktop/gradefolders.txt')):
        gradefolders = os.path.expanduser('~/Desktop/gradefolders.txt')
    else:
        print 'The gradefolders.txt file appears to be missing.'
        print 'gradefolders.txt should contain the name of one grade level per line'
        print 'staring with Preeschool through 12thin the format below:'
        print '\n00-Preeschool\n00-Transition Kindergarten\n00-zKindergarten...\n09-Grade\n10-Grade\n11-Grade\n12-Grade'
        print 'please create a gradefolders.txt file and place it on the desktop'
        print 'exiting'
        sys.exit(0)
    
    
if updateConfig:
    configuration.create_config(cfgfile, myConfig)




2018-08-16 19:58:20,828: [INFO: root.<module>] ===Starting portfolio_creator Log===
2018-08-16 19:58:20,829: [DEBUG: configuration.get_config] reading configuration file at: /Users/aaronciuffo/.config/portfolio_creator/portfolio_creator.ini
2018-08-16 19:58:20,831: [DEBUG: root.<module>] current configuration:
2018-08-16 19:58:20,832: [DEBUG: root.<module>] 
{'Main': {'folderid': '1NWVX-ZDTbT5yUT-Rw3a3fHQNptB4W4lb', 'credentials': '~/.config/portfolio_creator/credentials/', 'foldername': 'Portfolios', 'teamdrivename': 'IT Blabla', 'teamdriveid': '0ACLfU8KeD_BHUk9PVA'}}
Welcome to the portfolio creator for Google Team Drive version 00.00 - 18.08.13
This program will create portfolio folders in Google Team Drive for students.
You will need a student_export.text file from PowerSchool with at least the following information:
     ClassOf, FirstLast, Student_Number

The order of the CSV does not matter, but the headers must be on the very first line.
2018-08-16 19:58:20,833: [INFO: root.<mo

SystemExit: 0

/Users/aaronciuffo/Virtualenvs/python2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [184]:
os.path.isfile('./gradefolders.txt')

False

In [117]:
if folderID in ['F']:
    print 'yes!'
else:
    print 'no!'

no!
